# Local Landcover Classification Analysis for Uzbekistan

This notebook processes the comprehensive satellite tiles downloaded from Google Earth Engine and performs landcover classification using your training labels.

## Prerequisites:
1. Download the GeoTIFF files from Google Drive `earthengine_exports` folder
2. Copy them to `data/downloaded_tiles/` directory
3. Ensure `landcover_training.geojson` is in `data/training/` directory

## Step 1: Setup and Import Libraries

In [ ]:
# Import required libraries
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.features import geometry_mask
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Machine learning libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
import seaborn as sns

print("✅ All libraries imported successfully!")

## Step 2: Define Paths and Load Class Labels

In [ ]:
# Set up paths
project_root = Path.cwd().parent
data_dir = project_root / "data"
training_dir = data_dir / "training"
tiles_dir = data_dir / "downloaded_tiles"  # Copy tiles from Google Drive here
results_dir = data_dir / "results"
models_dir = data_dir / "models"

# Create directories if they don't exist
tiles_dir.mkdir(parents=True, exist_ok=True)
results_dir.mkdir(parents=True, exist_ok=True)
models_dir.mkdir(parents=True, exist_ok=True)

print(f"📂 Project root: {project_root}")
print(f"📂 Training data: {training_dir}")
print(f"📂 Tiles directory: {tiles_dir}")
print(f"📂 Results directory: {results_dir}")

# Define the 12 landcover classes based on your training data
LANDCOVER_CLASSES = {
    1: 'Residential',
    2: 'Agriculture', 
    3: 'Buildings',
    4: 'Forest',
    5: 'Residential_Private',
    6: 'Roads_Highways',
    7: 'Land_Stock',
    8: 'Non_Residential',
    9: 'Protected',
    10: 'Railways',
    11: 'Shared_Lands',
    12: 'Water'
}

print("\n🎯 Landcover Classes:")
for class_id, class_name in LANDCOVER_CLASSES.items():
    print(f"   {class_id:2d}. {class_name}")

## Step 3: Load Training Labels from GeoJSON

In [ ]:
# Load the training GeoJSON file
training_geojson = training_dir / "landcover_training.geojson"

if training_geojson.exists():
    print(f"📂 Loading training data: {training_geojson}")
    training_gdf = gpd.read_file(training_geojson)
    
    # Ensure CRS is set (assuming WGS84 if not specified)
    if training_gdf.crs is None:
        training_gdf.set_crs('EPSG:4326', inplace=True)
    
    print(f"✅ Training data loaded successfully!")
    print(f"   📊 Total features: {len(training_gdf):,}")
    print(f"   🗂️  Columns: {list(training_gdf.columns)}")
    print(f"   🌍 CRS: {training_gdf.crs}")
    
    # Analyze class distribution
    if 'layer_id' in training_gdf.columns:
        layer_counts = training_gdf['layer_id'].value_counts().sort_index()
        print(f"\n📈 Training Data Distribution:")
        for layer_id, count in layer_counts.items():
            if layer_id in LANDCOVER_CLASSES:
                print(f"   {layer_id:2d}. {LANDCOVER_CLASSES[layer_id]:18}: {count:6,} polygons")
else:
    print(f"❌ Training data not found: {training_geojson}")
    print("Please ensure landcover_training.geojson is in the training directory")

## Step 4: Load and Explore Satellite Tiles

In [ ]:
# Check for available tiles
print("🔍 Checking for downloaded tiles...")
print("\n📥 Please copy your downloaded GeoTIFF files from Google Drive to:")
print(f"   {tiles_dir}")
print("\nExpected files:")
print("   - uzbekistan_tile_recent_3_months_comprehensive.tif")
print("   - uzbekistan_tile_summer_2023_comprehensive.tif")
print("   - uzbekistan_tile_winter_2023_2024_comprehensive.tif")

# List available TIF files
tif_files = list(tiles_dir.glob("*.tif")) + list(tiles_dir.glob("*.tiff"))
if tif_files:
    print(f"\n✅ Found {len(tif_files)} tile(s):")
    for tif_file in tif_files:
        print(f"   - {tif_file.name}")
        
    # Select the first tile for analysis
    selected_tile = tif_files[0]
    print(f"\n📊 Selected for analysis: {selected_tile.name}")
else:
    print("\n⚠️  No TIF files found in the tiles directory.")
    print("   Please download and copy the tiles from Google Drive first.")

In [ ]:
def load_and_explore_tile(tile_path):
    """Load a satellite tile and display its properties"""
    with rasterio.open(tile_path) as src:
        print(f"\n📊 Tile Information: {tile_path.name}")
        print(f"   - Shape: {src.height} x {src.width} pixels")
        print(f"   - Bands: {src.count}")
        print(f"   - CRS: {src.crs}")
        print(f"   - Pixel Size: {src.res[0]}m x {src.res[1]}m")
        print(f"   - Bounds: {src.bounds}")
        
        # Expected band names based on our comprehensive export
        expected_bands = [
            'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7',
            'NDVI', 'NDWI', 'MNDWI', 'NDBI', 'EVI', 'SAVI',
            'elevation', 'SLOPE', 'ASPECT', 'HILLSHADE',
            'TERRAIN_MOUNTAIN', 'WATER_MASK',
            'VEG_SPARSE', 'VEG_MODERATE', 'VEG_DENSE',
            'TEMP_C', 'NIR_TEXTURE'
        ]
        
        print("\n   📊 Expected Bands (based on GEE export):")
        for i, name in enumerate(expected_bands[:src.count], 1):
            print(f"      Band {i:2d}: {name}")
        
        return src.meta, expected_bands[:src.count]

# Load and explore the selected tile
if tif_files:
    tile_meta, band_names = load_and_explore_tile(selected_tile)
    print(f"\n✅ Ready for analysis with {len(band_names)} bands")

## Step 5: Extract Training Samples from Satellite Data

In [ ]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import gc  # For garbage collection to manage memory

def clip_raster_by_polygon(raster_path, polygon_gdf, output_path=None):
    """
    Clip raster by polygon boundaries to reduce processing load.
    """
    import rasterio.mask
    
    with rasterio.open(raster_path) as src:
        # Get the polygon geometries
        geometries = polygon_gdf.geometry.values
        
        # Clip the raster
        clipped_data, clipped_transform = rasterio.mask.mask(
            src, geometries, crop=True, nodata=src.nodata
        )
        
        # Update metadata
        clipped_meta = src.meta.copy()
        clipped_meta.update({
            'height': clipped_data.shape[1],
            'width': clipped_data.shape[2],
            'transform': clipped_transform
        })
        
        if output_path:
            with rasterio.open(output_path, 'w', **clipped_meta) as dst:
                dst.write(clipped_data)
            return output_path, clipped_meta
        else:
            return clipped_data, clipped_meta, clipped_transform

def extract_pixel_values_efficient(raster_path, polygons_gdf, class_column='layer_id', 
                                 region_name="region", max_samples_per_class=5000):
    """
    Extract pixel values from raster for each polygon with memory optimization.
    Returns features (X) and labels (y) for training.
    """
    print(f"\n🔍 Extracting features from {region_name}...")
    
    features_list = []
    labels_list = []
    
    with rasterio.open(raster_path) as src:
        # Get raster properties
        n_bands = src.count
        print(f"   ? Processing {n_bands} bands")
        
        # Process each class separately to control memory usage
        unique_classes = polygons_gdf[class_column].unique()
        print(f"   🏷️  Found {len(unique_classes)} classes: {unique_classes}")
        
        for class_id in unique_classes:
            print(f"   Processing class {class_id}...")
            
            # Get polygons for this class
            class_polygons = polygons_gdf[polygons_gdf[class_column] == class_id]
            
            # Extract pixels for this class
            class_features = []
            
            for idx, polygon in class_polygons.iterrows():
                try:
                    # Create a mask for this polygon
                    geometry = [polygon.geometry]
                    masked_data, masked_transform = rasterio.mask.mask(
                        src, geometry, crop=True, nodata=src.nodata, filled=False
                    )
                    
                    # Get valid pixels (not nodata)
                    valid_mask = ~masked_data.mask[0]  # Use first band for mask
                    
                    if valid_mask.sum() > 0:  # If there are valid pixels
                        # Extract all bands for valid pixels
                        pixel_values = masked_data[:, valid_mask].T  # Shape: (n_pixels, n_bands)
                        
                        # Limit samples per polygon to manage memory
                        if len(pixel_values) > 1000:
                            indices = np.random.choice(len(pixel_values), 1000, replace=False)
                            pixel_values = pixel_values[indices]
                        
                        class_features.append(pixel_values)
                        
                except Exception as e:
                    print(f"     ⚠️  Error processing polygon {idx}: {e}")
                    continue
            
            if class_features:
                # Combine all features for this class
                class_features_combined = np.vstack(class_features)
                
                # Limit total samples per class
                if len(class_features_combined) > max_samples_per_class:
                    indices = np.random.choice(len(class_features_combined), 
                                             max_samples_per_class, replace=False)
                    class_features_combined = class_features_combined[indices]
                
                features_list.append(class_features_combined)
                labels_list.extend([class_id] * len(class_features_combined))
                
                print(f"     ✅ Extracted {len(class_features_combined)} samples for class {class_id}")
            
            # Clean up memory
            gc.collect()
    
    if features_list:
        X = np.vstack(features_list)
        y = np.array(labels_list)
        
        print(f"\n📊 Final dataset: {X.shape[0]} samples, {X.shape[1]} features")
        print(f"   Class distribution: {np.unique(y, return_counts=True)}")
        
        return X, y
    else:
        print("❌ No valid samples extracted!")
        return None, None

In [ ]:
class MultiTemporalRegionalClassifier:
    """
    A classifier that handles multiple time periods and regional processing.
    """
    
    def __init__(self, model_dir="models"):
        self.model_dir = Path(model_dir)
        self.model_dir.mkdir(exist_ok=True)
        self.models = {}
        self.band_names = None
        
    def train_regional_model(self, raster_paths, training_gdf, region_polygons=None,
                           model_name="regional_model", test_size=0.2):
        """
        Train a model using multiple raster files and regional clipping.
        
        Args:
            raster_paths: Dict with keys like {'recent': path, 'summer': path, 'winter': path}
            training_gdf: GeoDataFrame with training polygons
            region_polygons: Optional GeoDataFrame for regional clipping
            model_name: Name for saving the model
        """
        print(f"\n🎯 Training {model_name} model...")
        
        all_features = []
        all_labels = []
        
        # Process each time period
        for period, raster_path in raster_paths.items():
            print(f"\n📅 Processing {period} data...")
            
            # Clip to region if specified
            if region_polygons is not None:
                print("   🔧 Clipping raster to region...")
                clipped_data, clipped_meta, clipped_transform = clip_raster_by_polygon(
                    raster_path, region_polygons
                )
                
                # Create temporary clipped file
                temp_path = self.model_dir / f"temp_{period}_clipped.tif"
                with rasterio.open(temp_path, 'w', **clipped_meta) as dst:
                    dst.write(clipped_data)
                
                # Extract features from clipped raster
                X_period, y_period = extract_pixel_values_efficient(
                    temp_path, training_gdf, region_name=f"{period}_region"
                )
                
                # Clean up temporary file
                temp_path.unlink()
            else:
                # Extract features from full raster
                X_period, y_period = extract_pixel_values_efficient(
                    raster_path, training_gdf, region_name=period
                )
            
            if X_period is not None:
                all_features.append(X_period)
                all_labels.append(y_period)
        
        if not all_features:
            print("❌ No features extracted from any time period!")
            return None
        
        # Combine all time periods
        print("\n🔄 Combining multi-temporal features...")
        X_combined = np.vstack(all_features)
        y_combined = np.hstack(all_labels)
        
        print(f"📊 Combined dataset: {X_combined.shape[0]} samples, {X_combined.shape[1]} features")
        
        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X_combined, y_combined, test_size=test_size, random_state=42, stratify=y_combined
        )
        
        # Train Random Forest model
        print("\n🌲 Training Random Forest classifier...")
        from sklearn.ensemble import RandomForestClassifier
        
        rf_model = RandomForestClassifier(
            n_estimators=100,
            max_depth=20,
            min_samples_split=10,
            min_samples_leaf=5,
            random_state=42,
            n_jobs=-1,
            verbose=1
        )
        
        rf_model.fit(X_train, y_train)
        
        # Evaluate model
        print("\n📈 Model evaluation...")
        train_score = rf_model.score(X_train, y_train)
        test_score = rf_model.score(X_test, y_test)
        
        print(f"   Training accuracy: {train_score:.3f}")
        print(f"   Testing accuracy: {test_score:.3f}")
        
        # Detailed classification report
        y_pred = rf_model.predict(X_test)
        print("\n📋 Classification Report:")
        print(classification_report(y_test, y_pred))
        
        # Save model
        model_path = self.model_dir / f"{model_name}.pkl"
        with open(model_path, 'wb') as f:
            pickle.dump({
                'model': rf_model,
                'band_names': self.band_names,
                'class_names': training_gdf['layer_id'].unique(),
                'training_info': {
                    'train_samples': len(X_train),
                    'test_samples': len(X_test),
                    'train_accuracy': train_score,
                    'test_accuracy': test_score
                }
            }, f)
        
        print(f"💾 Model saved to: {model_path}")
        
        # Store in memory
        self.models[model_name] = rf_model
        
        return rf_model, test_score
    
    def load_model(self, model_name):
        """Load a saved model"""
        model_path = self.model_dir / f"{model_name}.pkl"
        
        if model_path.exists():
            with open(model_path, 'rb') as f:
                model_data = pickle.load(f)
            
            self.models[model_name] = model_data['model']
            self.band_names = model_data['band_names']
            
            print(f"✅ Model {model_name} loaded successfully")
            print(f"   Training accuracy: {model_data['training_info']['train_accuracy']:.3f}")
            print(f"   Testing accuracy: {model_data['training_info']['test_accuracy']:.3f}")
            
            return model_data['model']
        else:
            print(f"❌ Model file not found: {model_path}")
            return None
    
    def classify_large_raster(self, raster_path, model_name, output_path, 
                            chunk_size=1024, region_polygons=None):
        """
        Classify a large raster using chunked processing to manage memory.
        """
        print(f"\n🗺️  Classifying large raster: {raster_path}")
        
        if model_name not in self.models:
            self.load_model(model_name)
        
        if model_name not in self.models:
            print(f"❌ Model {model_name} not available!")
            return None
        
        model = self.models[model_name]
        
        with rasterio.open(raster_path) as src:
            # Get raster properties
            profile = src.profile.copy()
            profile.update(dtype=rasterio.uint8, count=1, compress='lzw')
            
            height, width = src.height, src.width
            
            print(f"   📊 Raster size: {height} x {width} pixels")
            print(f"   🔧 Processing in {chunk_size}x{chunk_size} chunks...")
            
            # Create output raster
            with rasterio.open(output_path, 'w', **profile) as dst:
                # Process in chunks
                for row in range(0, height, chunk_size):
                    for col in range(0, width, chunk_size):
                        # Define window
                        window = rasterio.windows.Window(
                            col, row, 
                            min(chunk_size, width - col),
                            min(chunk_size, height - row)
                        )
                        
                        # Read chunk
                        chunk_data = src.read(window=window)  # Shape: (bands, height, width)
                        
                        # Reshape for prediction
                        chunk_height, chunk_width = chunk_data.shape[1], chunk_data.shape[2]
                        chunk_reshaped = chunk_data.reshape(chunk_data.shape[0], -1).T
                        
                        # Handle nodata values
                        valid_mask = ~np.isnan(chunk_reshaped).any(axis=1)
                        
                        if valid_mask.sum() > 0:
                            # Predict only valid pixels
                            valid_pixels = chunk_reshaped[valid_mask]
                            predictions = model.predict(valid_pixels)
                            
                            # Create output chunk
                            output_chunk = np.full(chunk_height * chunk_width, 0, dtype=np.uint8)
                            output_chunk[valid_mask] = predictions
                            output_chunk = output_chunk.reshape(chunk_height, chunk_width)
                        else:
                            output_chunk = np.zeros((chunk_height, chunk_width), dtype=np.uint8)
                        
                        # Write chunk
                        dst.write(output_chunk[np.newaxis, :, :], window=window)
                        
                        print(f"     ✅ Processed chunk ({row}:{row+chunk_height}, {col}:{col+chunk_width})")
        
        print(f"🎉 Classification complete! Saved to: {output_path}")
        return output_path

## Step 6: Regional Processing Implementation

Now let's implement the regional processing strategy for your large territory:

In [ ]:
# Initialize the regional classifier
classifier = MultiTemporalRegionalClassifier(model_dir="models")

# Load regional boundaries (Tashkent borders for regional processing)
print("🗺️  Loading regional boundaries...")
tashkent_borders = gpd.read_file("../data/training/Tashkent_borders.geojson")
print(f"   ✅ Loaded {len(tashkent_borders)} boundary polygons")

# Define your three raster files (update paths as needed)
raster_files = {
    'recent_3_months': "../data/downloaded_tiles/uzbekistan_tile_recent_3_months_comprehensive.tif",
    'summer_2023': "../data/downloaded_tiles/uzbekistan_tile_summer_2023_comprehensive.tif", 
    'winter_2023_2024': "../data/downloaded_tiles/uzbekistan_tile_winter_2023_2024_comprehensive.tif"
}

# Check which files exist
available_rasters = {}
for period, path in raster_files.items():
    if Path(path).exists():
        available_rasters[period] = path
        print(f"✅ Found {period}: {Path(path).name}")
    else:
        print(f"❌ Missing {period}: {path}")

print(f"\n📊 Available for processing: {len(available_rasters)} raster files")

# Store band names for reference
if available_rasters:
    # Use the first available raster to get band information
    first_raster = list(available_rasters.values())[0]
    classifier.band_names = [
        'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7',
        'NDVI', 'NDWI', 'MNDWI', 'NDBI', 'EVI', 'SAVI',
        'elevation', 'SLOPE', 'ASPECT', 'HILLSHADE',
        'TERRAIN_MOUNTAIN', 'WATER_MASK',
        'VEG_SPARSE', 'VEG_MODERATE', 'VEG_DENSE',
        'TEMP_C', 'NIR_TEXTURE'
    ]

In [ ]:
# STRATEGY 1: Train on selected regions first
print("\n🎯 STRATEGY 1: Regional Model Training")
print("="*50)

if available_rasters and training_data is not None:
    # Option A: Train on a subset of Tashkent (faster, good for testing)
    print("\n📍 Option A: Training on Tashkent region subset...")
    
    # You can modify this to select specific regions or use a bounding box
    # For now, let's use the first polygon as a training region
    training_region = tashkent_borders.iloc[:1].copy()  # First polygon only
    
    try:
        # Train the regional model
        model, accuracy = classifier.train_regional_model(
            raster_paths=available_rasters,
            training_gdf=training_data,
            region_polygons=training_region,
            model_name="tashkent_regional_model"
        )
        
        if model is not None:
            print(f"\n✅ Regional model trained successfully!")
            print(f"   🎯 Accuracy: {accuracy:.3f}")
            print(f"   💾 Model saved for reuse")
            
            # Show feature importance
            if hasattr(model, 'feature_importances_'):
                feature_importance = pd.DataFrame({
                    'feature': classifier.band_names,
                    'importance': model.feature_importances_
                }).sort_values('importance', ascending=False)
                
                print("\n🏆 Top 10 Most Important Features:")
                print(feature_importance.head(10).to_string(index=False))
        
    except Exception as e:
        print(f"❌ Error in regional training: {e}")
        print("   💡 This might be due to memory constraints or data issues")

else:
    print("⚠️  Skipping training - missing raster files or training data")
    print("   Please ensure you have downloaded the GeoTIFF files from Google Drive")

In [ ]:
# STRATEGY 2: Apply trained model to entire territory
print("\n🗺️  STRATEGY 2: Large-Scale Classification")
print("="*50)

# Create output directory
output_dir = Path("../data/results/classifications")
output_dir.mkdir(parents=True, exist_ok=True)

# Check if we have a trained model
model_path = Path("models/tashkent_regional_model.pkl")
if model_path.exists() or "tashkent_regional_model" in classifier.models:
    print("✅ Trained model available for classification")
    
    # Classify each time period
    for period, raster_path in available_rasters.items():
        print(f"\n📅 Classifying {period} data...")
        
        output_path = output_dir / f"landcover_classification_{period}.tif"
        
        try:
            # Use chunked processing for large rasters
            result_path = classifier.classify_large_raster(
                raster_path=raster_path,
                model_name="tashkent_regional_model",
                output_path=str(output_path),
                chunk_size=512,  # Adjust based on your memory capacity
                region_polygons=None  # Set to tashkent_borders if you want to clip to region
            )
            
            if result_path:
                print(f"✅ Classification saved: {result_path}")
                
                # Quick stats about the classification
                with rasterio.open(result_path) as src:
                    classified_data = src.read(1)
                    unique_classes, counts = np.unique(classified_data[classified_data > 0], return_counts=True)
                    
                    print(f"   📊 Classification Statistics for {period}:")
                    for class_id, count in zip(unique_classes, counts):
                        percentage = (count / counts.sum()) * 100
                        print(f"      Class {class_id}: {count:,} pixels ({percentage:.1f}%)")
            
        except Exception as e:
            print(f"❌ Error classifying {period}: {e}")
            print("   💡 Try reducing chunk_size or using regional clipping")

else:
    print("⚠️  No trained model found!")
    print("   Please run the training step first or load an existing model")
    
    # Show how to load an existing model
    print("\n💡 To load an existing model:")
    print("   classifier.load_model('tashkent_regional_model')")

In [ ]:
# STRATEGY 3: Multi-temporal analysis and visualization
print("\n📊 STRATEGY 3: Multi-temporal Analysis")
print("="*50)

def visualize_classification_results():
    """Visualize and compare classification results across time periods"""
    
    # Look for classification results
    classification_files = list(output_dir.glob("landcover_classification_*.tif"))
    
    if not classification_files:
        print("❌ No classification results found!")
        print("   Run the classification step first")
        return
    
    print(f"📈 Found {len(classification_files)} classification results")
    
    # Create a figure for comparison
    n_files = len(classification_files)
    fig, axes = plt.subplots(1, n_files, figsize=(6*n_files, 6))
    if n_files == 1:
        axes = [axes]
    
    # Define class colors (you can customize these)
    class_colors = {
        1: '#8B4513',  # Residential - Brown
        2: '#FFD700',  # Agriculture - Gold
        3: '#696969',  # Buildings - Dark Gray
        4: '#228B22',  # Forest - Forest Green
        5: '#DDA0DD',  # Residential_Private - Plum
        6: '#2F4F4F',  # Roads_Highways - Dark Slate Gray
        7: '#D2B48C',  # Land_Stock - Tan
        8: '#800080',  # Non_Residential - Purple
        9: '#32CD32',  # Protected - Lime Green
        10: '#A0A0A0', # Railways - Gray
        11: '#90EE90', # Shared_Lands - Light Green
        12: '#4169E1'  # Water - Royal Blue
    }
    
    for i, file_path in enumerate(classification_files):
        period = file_path.stem.replace('landcover_classification_', '')
        
        with rasterio.open(file_path) as src:
            classified_data = src.read(1)
            
            # Create custom colormap
            from matplotlib.colors import ListedColormap
            colors = [class_colors.get(i, '#000000') for i in range(13)]
            cmap = ListedColormap(colors)
            
            # Plot
            im = axes[i].imshow(classified_data, cmap=cmap, vmin=0, vmax=12)
            axes[i].set_title(f'{period.replace("_", " ").title()}', fontsize=12)
            axes[i].axis('off')
            
            # Add statistics
            unique_classes, counts = np.unique(classified_data[classified_data > 0], return_counts=True)
            total_pixels = counts.sum()
            
            stats_text = f"Classes: {len(unique_classes)}\nPixels: {total_pixels:,}"
            axes[i].text(0.02, 0.98, stats_text, transform=axes[i].transAxes, 
                        verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", 
                        facecolor="white", alpha=0.8))
    
    plt.tight_layout()
    plt.savefig(output_dir / "multi_temporal_comparison.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    return classification_files

def analyze_temporal_changes(classification_files):
    """Analyze changes between different time periods"""
    
    if len(classification_files) < 2:
        print("⚠️  Need at least 2 time periods for change analysis")
        return
    
    print("\n🔄 Temporal Change Analysis")
    print("-" * 30)
    
    # Load all classifications
    classifications = {}
    for file_path in classification_files:
        period = file_path.stem.replace('landcover_classification_', '')
        with rasterio.open(file_path) as src:
            classifications[period] = src.read(1)
    
    # Compare each pair of time periods
    periods = list(classifications.keys())
    for i in range(len(periods)-1):
        period1, period2 = periods[i], periods[i+1]
        
        print(f"\n📅 Comparing {period1} → {period2}")
        
        data1 = classifications[period1]
        data2 = classifications[period2]
        
        # Calculate change matrix
        valid_mask = (data1 > 0) & (data2 > 0)
        changes = (data1[valid_mask] != data2[valid_mask]).sum()
        total_valid = valid_mask.sum()
        
        change_percentage = (changes / total_valid) * 100 if total_valid > 0 else 0
        
        print(f"   🔄 Changed pixels: {changes:,} ({change_percentage:.1f}%)")
        print(f"   ✅ Stable pixels: {total_valid - changes:,} ({100-change_percentage:.1f}%)")
        
        # Find most common changes
        change_pairs = []
        for old_class in np.unique(data1[valid_mask]):
            for new_class in np.unique(data2[valid_mask]):
                if old_class != new_class:
                    change_count = ((data1 == old_class) & (data2 == new_class)).sum()
                    if change_count > 0:
                        change_pairs.append((old_class, new_class, change_count))
        
        # Sort by frequency and show top changes
        change_pairs.sort(key=lambda x: x[2], reverse=True)
        print(f"   🏆 Top 5 land cover changes:")
        for old_c, new_c, count in change_pairs[:5]:
            print(f"      Class {old_c} → Class {new_c}: {count:,} pixels")

# Run the visualization and analysis
classification_results = visualize_classification_results()
if classification_results:
    analyze_temporal_changes(classification_results)

In [ ]:
# Extract training samples if we have both tiles and training data
if tif_files and 'training_gdf' in locals():
    print("🚀 Starting feature extraction...")
    X, y = extract_pixel_values(selected_tile, training_gdf)
    
    if X is not None:
        # Remove any pixels with NaN or infinite values
        valid_pixels = ~np.any(np.isnan(X) | np.isinf(X), axis=1)
        X = X[valid_pixels]
        y = y[valid_pixels]
        
        print(f"\n📊 Final dataset after cleaning:")
        print(f"   - Features shape: {X.shape}")
        print(f"   - Labels shape: {y.shape}")
        print(f"   - Unique classes: {sorted(np.unique(y))}")
        
        # Show class distribution
        print("\n📈 Class distribution in extracted pixels:")
        for class_id in sorted(np.unique(y)):
            count = np.sum(y == class_id)
            percentage = (count / len(y)) * 100
            class_name = LANDCOVER_CLASSES.get(class_id, f"Unknown_{class_id}")
            print(f"   {class_id:2d}. {class_name:18}: {count:8,} pixels ({percentage:5.2f}%)")
else:
    print("⚠️  Missing tiles or training data. Please ensure both are available.")

## Step 6: Train Random Forest Classifier

In [ ]:
if 'X' in locals() and X is not None:
    # Split data into training and testing sets
    print("🔄 Splitting data into train/test sets...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    print(f"   Training set: {X_train.shape[0]:,} pixels")
    print(f"   Test set: {X_test.shape[0]:,} pixels")
    
    # Train Random Forest classifier
    print("\n🌲 Training Random Forest Classifier...")
    rf_classifier = RandomForestClassifier(
        n_estimators=100,
        max_depth=20,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1,
        verbose=1
    )
    
    rf_classifier.fit(X_train, y_train)
    print("✅ Model training complete!")
    
    # Evaluate on test set
    y_pred = rf_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\n📊 Test Accuracy: {accuracy:.4f}")
    
    # Save the model
    model_path = models_dir / "rf_landcover_classifier.joblib"
    joblib.dump(rf_classifier, model_path)
    print(f"\n💾 Model saved to: {model_path}")

## Step 7: Evaluate Model Performance

In [ ]:
if 'rf_classifier' in locals():
    # Generate classification report
    print("\n📊 Classification Report:")
    print("="*60)
    
    # Create label names for the report
    unique_classes = sorted(np.unique(y_test))
    class_names = [LANDCOVER_CLASSES[i] for i in unique_classes]
    
    report = classification_report(
        y_test, y_pred,
        labels=unique_classes,
        target_names=class_names,
        digits=3
    )
    print(report)
    
    # Feature importance
    feature_importance = rf_classifier.feature_importances_
    
    # Create feature importance dataframe
    importance_df = pd.DataFrame({
        'feature': band_names,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
    print("\n🎯 Top 10 Most Important Features:")
    print("="*40)
    for idx, row in importance_df.head(10).iterrows():
        print(f"{row['feature']:20} : {row['importance']:.4f}")

In [ ]:
# Plot confusion matrix
if 'rf_classifier' in locals():
    cm = confusion_matrix(y_test, y_pred, labels=unique_classes)
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        cm, 
        annot=True, 
        fmt='d', 
        cmap='Blues',
        xticklabels=class_names,
        yticklabels=class_names
    )
    plt.title('Confusion Matrix - Landcover Classification', fontsize=16)
    plt.xlabel('Predicted Class', fontsize=12)
    plt.ylabel('True Class', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Save the plot
    plt.savefig(results_dir / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n💾 Confusion matrix saved to: {results_dir / 'confusion_matrix.png'}")

## Step 8: Apply Model to Create Landcover Map

In [ ]:
def classify_raster(raster_path, model, output_path, chunk_size=1000):
    """
    Apply the trained model to classify the entire raster.
    Process in chunks to manage memory.
    """
    with rasterio.open(raster_path) as src:
        # Get metadata for output
        meta = src.meta.copy()
        meta.update({
            'dtype': 'uint8',
            'count': 1,
            'compress': 'lzw'
        })
        
        print(f"📊 Processing raster: {src.height} x {src.width} pixels")
        
        # Create output raster
        with rasterio.open(output_path, 'w', **meta) as dst:
            # Process in chunks
            total_chunks = (src.height + chunk_size - 1) // chunk_size
            
            for i, row_start in enumerate(range(0, src.height, chunk_size)):
                # Calculate chunk boundaries
                row_end = min(row_start + chunk_size, src.height)
                
                print(f"   Processing chunk {i+1}/{total_chunks}: rows {row_start}-{row_end}")
                
                # Read chunk
                window = rasterio.windows.Window(
                    0, row_start, 
                    src.width, row_end - row_start
                )
                chunk_data = src.read(window=window)
                
                # Reshape for prediction (pixels x bands)
                n_bands, n_rows, n_cols = chunk_data.shape
                chunk_reshaped = chunk_data.transpose(1, 2, 0).reshape(-1, n_bands)
                
                # Handle NaN and infinite values
                valid_pixels = ~np.any(np.isnan(chunk_reshaped) | np.isinf(chunk_reshaped), axis=1)
                predictions = np.zeros(chunk_reshaped.shape[0], dtype=np.uint8)
                
                if np.any(valid_pixels):
                    # Predict only on valid pixels
                    predictions[valid_pixels] = model.predict(chunk_reshaped[valid_pixels])
                
                # Reshape back and write
                predictions_2d = predictions.reshape(n_rows, n_cols)
                dst.write(predictions_2d, 1, window=window)
    
    print(f"\n✅ Classification complete! Output saved to: {output_path}")

In [ ]:
# Apply the model to create a landcover map
if 'rf_classifier' in locals() and tif_files:
    print("🗺️  Creating landcover classification map...")
    
    # Define output path
    output_map = results_dir / f"landcover_map_{selected_tile.stem}.tif"
    
    # Classify the raster
    classify_raster(selected_tile, rf_classifier, output_map)
    
    print(f"\n🎉 Landcover map created successfully!")
    print(f"   📍 Location: {output_map}")
else:
    print("⚠️  Model or tiles not available for classification.")

## Step 9: Visualize Results

In [ ]:
# Visualize a sample of the classification result
if 'output_map' in locals() and output_map.exists():
    with rasterio.open(output_map) as src:
        # Read a sample area (center of the image)
        h, w = src.height, src.width
        sample_size = min(2000, h, w)
        
        window = rasterio.windows.Window(
            (w - sample_size) // 2,
            (h - sample_size) // 2,
            sample_size,
            sample_size
        )
        
        sample_data = src.read(1, window=window)
        
        # Create color map
        from matplotlib.colors import ListedColormap
        colors = [
            '#8B0000',  # 1. Residential (Dark Red)
            '#32CD32',  # 2. Agriculture (Lime Green)
            '#A0522D',  # 3. Buildings (Sienna)
            '#228B22',  # 4. Forest (Forest Green)
            '#FF6347',  # 5. Residential_Private (Tomato)
            '#696969',  # 6. Roads_Highways (Dim Gray)
            '#DEB887',  # 7. Land_Stock (Burlywood)
            '#800080',  # 8. Non_Residential (Purple)
            '#90EE90',  # 9. Protected (Light Green)
            '#2F4F4F',  # 10. Railways (Dark Slate Gray)
            '#F4A460',  # 11. Shared_Lands (Sandy Brown)
            '#4169E1'   # 12. Water (Royal Blue)
        ]
        
        cmap = ListedColormap(colors[:len(LANDCOVER_CLASSES)])
        
        # Plot
        fig, ax = plt.subplots(figsize=(15, 12))
        im = ax.imshow(sample_data, cmap=cmap, vmin=1, vmax=len(LANDCOVER_CLASSES))
        ax.set_title('Landcover Classification Sample (Center Region)', fontsize=16)
        ax.axis('off')
        
        # Add colorbar with class labels
        cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04, shrink=0.8)
        cbar.set_ticks(list(LANDCOVER_CLASSES.keys()))
        cbar.set_ticklabels([f"{k}. {v}" for k, v in LANDCOVER_CLASSES.items()])
        cbar.ax.tick_params(labelsize=10)
        
        plt.tight_layout()
        plt.savefig(results_dir / 'landcover_sample_visualization.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"\n💾 Visualization saved to: {results_dir / 'landcover_sample_visualization.png'}")

## Step 10: Summary and Next Steps

In [ ]:
print("="*70)
print("🎉 LANDCOVER CLASSIFICATION ANALYSIS COMPLETE!")
print("="*70)

if 'rf_classifier' in locals():
    print(f"\n📊 Model Performance:")
    print(f"   • Test Accuracy: {accuracy:.4f}")
    print(f"   • Number of training pixels: {X_train.shape[0]:,}")
    print(f"   • Number of features: {X_train.shape[1]}")
    print(f"   • Number of classes: {len(np.unique(y))}")

print(f"\n📁 Output Files:")
output_files = [
    (models_dir / "rf_landcover_classifier.joblib", "Trained Model"),
    (results_dir / f"landcover_map_{selected_tile.stem}.tif", "Landcover Map"),
    (results_dir / 'confusion_matrix.png', "Confusion Matrix"),
    (results_dir / 'landcover_sample_visualization.png', "Sample Visualization")
]

for file_path, description in output_files:
    if file_path.exists():
        print(f"   ✅ {description}: {file_path}")
    else:
        print(f"   ❌ {description}: Not created")

print(f"\n🚀 Next Steps:")
print("   1. Review the confusion matrix to identify classes that need improvement")
print("   2. Analyze feature importance to understand which bands are most useful")
print("   3. Consider adding more training data for underrepresented classes")
print("   4. Apply the model to other seasonal tiles for temporal analysis")
print("   5. Export results to GIS software (QGIS, ArcGIS) for further analysis")
print("   6. Consider hyperparameter tuning or ensemble methods for improved accuracy")
print("   7. Validate results with ground truth data if available")

print("="*70)